# Mobile-Mask-RCNN: training on Coco Dataset

In [1]:
"""
Based on the work of Waleed Abdulla (Matterport)
written by github.com/wozhouh
"""

# Import Python Packages
import argparse
import os
import imgaug

# Root directory of the project
ROOT_DIR = os.path.abspath("../")
import sys

# Import Mask RCNN
sys.path.append(ROOT_DIR)
from mrcnn import model as modellib

# Import COCO config
sys.path.append(os.path.join(ROOT_DIR, "samples", "coco"))  # To find local version
import coco

# # assign the GPU for training
# os.environ["CUDA_VISIBLE_DEVICES"] = "4"

Using TensorFlow backend.


In [2]:
# Default setting
HOME_DIR = os.getenv('HOME')
DEFAULT_WEIGHTS_DIR = os.path.join(ROOT_DIR, "weights")
DEFAULT_DATASET_YEAR = "2017"
DEFAULT_COCO_PATH = os.path.join(HOME_DIR, "data", "Coco")


# self-define
# MODEL_PATH_UNDER_HOME = "code/HumanMask/my-Mobile-Mask-RCNN/weights/init/mobilenet_1_0_224_tf_no_top.h5"
MODEL_PATH_UNDER_HOME = os.path.join(DEFAULT_WEIGHTS_DIR, "init", "mobilenet_1_0_224_tf_no_top.h5")

INIT_MODEL_PATH = os.path.join(HOME_DIR, MODEL_PATH_UNDER_HOME)

In [3]:
# training config
class TrainingConfig(coco.CocoConfig):
    NAME = "training"
    
    # GPU
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 2
    IMAGES_PER_GPU = 2
    
    # data
    IMAGE_MIN_DIM = 800
    IMAGE_MAX_DIM = 1024
    RPN_ANCHOR_SCALES = (32, 64, 128, 256, 512)
    
    # model
    BACKBONE = "mobilenetv1"
    BACKBONE_STRIDES = [4, 8, 16, 32, 64]
    
    # heads
    FPN_CLASSIF_FC_LAYERS_SIZE = 1024
    TOP_DOWN_PYRAMID_SIZE = 256
    
    # training
    LEARNING_RATE = 0.001
    WEIGHT_DECAY = 0.0001
    LOSS_WEIGHTS = {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
    STEPS_PER_EPOCH = 1000
    VALIDATION_STEPS = 50
    TRAIN_BN = False
    TRAIN_ROIS_PER_IMAGE = 200
    ROI_POSITIVE_RATIO = 0.33
    
config = TrainingConfig()
config.display()
model = modellib.MaskRCNN(mode="training", model_dir=DEFAULT_WEIGHTS_DIR, config=config)


Configurations:
BACKBONE                       mobilenetv1
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     4
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      2
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 2
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                93
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE       

In [4]:
# print the model summary
model.keras_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_image (InputLayer)        (None, None, None, 3 0                                            
__________________________________________________________________________________________________
input_image_meta (InputLayer)   (None, 93)           0                                            
__________________________________________________________________________________________________
input_rpn_match (InputLayer)    (None, None, 1)      0                                            
__________________________________________________________________________________________________
input_rpn_bbox (InputLayer)     (None, None, 4)      0                                            
__________________________________________________________________________________________________
input_gt_c

In [5]:
# Load the weights
model.load_weights(INIT_MODEL_PATH, by_name=True)

In [6]:
# Load the dataset
train_dataset = coco.CocoDataset()
train_dataset.load_coco(DEFAULT_COCO_PATH, "train", year=DEFAULT_DATASET_YEAR)
train_dataset.prepare()
val_dataset = coco.CocoDataset()
val_dataset.load_coco(DEFAULT_COCO_PATH, "val", year=DEFAULT_DATASET_YEAR)
val_dataset.prepare()

loading annotations into memory...
Done (t=14.98s)
creating index...
index created!
loading annotations into memory...
Done (t=4.42s)
creating index...
index created!


In [8]:
# Image Augmentation
# Right/Left flip 50% of the time
augmentation = imgaug.augmenters.Fliplr(0.5)

# # config the epoch
# epoch = train_dataset.dataset_size // (config.STEPS_PER_EPOCH * config.BATCH_SIZE)
# size of 'train2017' is 118287 and size of 'val2017' is 5000

In [9]:
# stage 0: warm up
# lr: 0.00002
# batch: 5
# epochs: 0.875
# layers: 'all'
# actually stopped at epoch 21
model.train(train_dataset=train_dataset,
                val_dataset=val_dataset,
                learning_rate=config.LEARNING_RATE / 50.0,
                epochs=30,
                layers='all',
                augmentation=augmentation)


Starting at epoch 0. LR=1e-05

Checkpoint Path: /home/processyuan/code/HumanMask/my-Mobile-Mask-RCNN/weights/training20181122T2313/mask_rcnn_training_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv_dw_1   (TimeDistributed)
mrcnn_mask_conv_dw_1_bn   (TimeDistributed)
mrcnn_mask_conv_pw_1   (TimeDistributed)
mrcnn_mask_conv_pw_1_bn   (TimeDistributed)
mrcnn_mask_conv_dw_2   (TimeDistributed)
mrcnn_mask_conv_dw_2_bn   (TimeDistributed)
mrcnn_mask_conv_pw_2   (TimeDistributed)
mrcnn_mask_conv_pw_2_bn   (TimeDistributed)
mrcnn_mask_conv_dw_3   (TimeDistributed)
mrcnn_mask_conv_dw_3_bn   (TimeDistributed

/home/processyuan/anaconda3/envs/tflite/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/home/processyuan/anaconda3/envs/tflite/lib/python3.6/site-packages/keras/engine/training_generator.py:47: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


Epoch 1/1
  36/1000 [>.............................] - ETA: 1:18:46 - loss: 8.2185 - rpn_class_loss: 1.7061 - rpn_bbox_loss: 0.8767 - mrcnn_class_loss: 4.3093 - mrcnn_bbox_loss: 0.6910 - mrcnn_mask_loss: 0.6354

Process ForkPoolWorker-68:
Process ForkPoolWorker-54:
Process ForkPoolWorker-56:
Process ForkPoolWorker-92:
Traceback (most recent call last):
Process ForkPoolWorker-64:
Process ForkPoolWorker-74:
Process ForkPoolWorker-58:
Process ForkPoolWorker-60:
  File "/home/processyuan/anaconda3/envs/tflite/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/processyuan/anaconda3/envs/tflite/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/processyuan/anaconda3/envs/tflite/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
Traceback (most recent call last):
  File "/home/processyuan/anaconda3/envs/tflite/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/home/processyuan/anaconda3/envs/tflite/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Process ForkPoolWorker-5:
Process ForkPoolWorke

Process ForkPoolWorker-6:
  File "/home/processyuan/anaconda3/envs/tflite/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/home/processyuan/anaconda3/envs/tflite/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/processyuan/anaconda3/envs/tflite/lib/python3.6/multiprocessing/pool.py", line 125, in worker
    put((job, i, result))
Process ForkPoolWorker-13:
  File "/home/processyuan/anaconda3/envs/tflite/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/processyuan/anaconda3/envs/tflite/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
Traceback (most recent call last):
  File "/home/processyuan/anaconda3/envs/tflite/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/home/processyuan/anaconda3/envs/tflite/lib/python3.6/multiprocessing/queues.py", lin

...

  File "/home/processyuan/anaconda3/envs/tflite/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/processyuan/anaconda3/envs/tflite/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/processyuan/anaconda3/envs/tflite/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/processyuan/anaconda3/envs/tflite/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/processyuan/anaconda3/envs/tflite/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/processyuan/anaconda3/envs/tflite/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/processyuan/anaconda3/envs/tflite/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/processyuan/anaconda3/envs/tfl

  File "/home/processyuan/anaconda3/envs/tflite/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/processyuan/anaconda3/envs/tflite/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/processyuan/anaconda3/envs/tflite/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/processyuan/anaconda3/envs/tflite/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/processyuan/anaconda3/envs/tflite/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/processyuan/anaconda3/envs/tflite/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/processyuan/anaconda3/envs/tflite/lib/python3.6/multiprocessing/proc

  File "/home/processyuan/anaconda3/envs/tflite/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/home/processyuan/anaconda3/envs/tflite/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/home/processyuan/anaconda3/envs/tflite/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/home/processyuan/anaconda3/envs/tflite/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/home/processyuan/anaconda3/envs/tflite/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/home/processyuan/anaconda3/envs/tflite/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/home/processyuan/anaconda3/envs/tflite/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/home/processyuan/anaconda3/envs/tflite/lib/python3.6/multiprocessing/queues.py", line 334, i

  File "/home/processyuan/anaconda3/envs/tflite/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/home/processyuan/anaconda3/envs/tflite/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/home/processyuan/anaconda3/envs/tflite/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/home/processyuan/anaconda3/envs/tflite/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/home/processyuan/anaconda3/envs/tflite/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/home/processyuan/anaconda3/envs/tflite/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/home/processyuan/anaconda3/envs/tflite/lib/python3.6/multiprocessin

  File "/home/processyuan/anaconda3/envs/tflite/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/home/processyuan/anaconda3/envs/tflite/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/processyuan/anaconda3/envs/tflite/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/home/processyuan/anaconda3/envs/tflite/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/home/processyuan/anaconda3/envs/tflite/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/home/processyuan/anaconda3/envs/tflite/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
  File "/home/processyuan/anaconda3/envs/tflite/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
KeyboardI

In [ ]:
# stage 1: heads-0
# lr: 0.001
# batch: 12
# epochs: (0.875+)1.116
# layers: 'heads'
# actually stopped at epoch 32
model.train(train_dataset=train_dataset,
                val_dataset=val_dataset,
                learning_rate=config.LEARNING_RATE,
                epochs=52,
                layers='heads',
                augmentation=augmentation)

In [ ]:
# stage 2: heads-1
# lr: 0.0002
# batch: 4
# epochs: (0.875+1.116+)2.029
# layers: 'heads'
# actually stopped at epoch 92
model.train(train_dataset=train_dataset,
                val_dataset=val_dataset,
                learning_rate=config.LEARNING_RATE / 5.0,
                epochs=92,
                layers='heads',
                augmentation=augmentation)